In [1]:
!pip install transformers datasets
!pip install librosa

In [2]:
import os
import torch
from transformers import pipeline
from datasets import load_dataset, Audio
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel

/Users/nm/opt/anaconda3/envs/nlp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-12-01 05:44:25.028825: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Text classification (sentiment analysis)

In [3]:
# The pipeline() is the easiest and fastest way to use a pretrained model for inference
# One can use the pipeline() out-of-the-box for many tasks across different modalities
classifier = pipeline("sentiment-analysis")
results = classifier(["We are very happy to show you the 🤗 Transformers library.", "We hope you don't hate it."])
for result in results:
    print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


label: POSITIVE, with score: 0.9998
label: NEGATIVE, with score: 0.5309


# Speech recognition

In [4]:
speech_recognizer = pipeline("automatic-speech-recognition", model="facebook/wav2vec2-base-960h")
dataset = load_dataset("PolyAI/minds14", name="en-US", split="train")

# need to make sure the sampling rate of the dataset matches the sampling rate \ 
# facebook/wav2vec2-base-960h was trained on
dataset = dataset.cast_column("audio", Audio(sampling_rate=speech_recognizer.feature_extractor.sampling_rate))

# audio files are automatically loaded and resampled when calling the "audio" column
# Extract the raw waveform arrays from the first 4 samples and pass it as a list to the pipeline
# NB: For larger datasets where the inputs are big (like in speech or vision), you’ll want to pass a generator \
# instead of a list to load all the inputs in memory
result = speech_recognizer(dataset[:4]["audio"])
print([d["text"] for d in result])

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_v', 'wav2vec2.encoder.pos_conv_embed.conv.weight_g']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.masked_spec_embed', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You sho

Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
['I WOULD LIKE TO SET UP A JOINT ACCOUNT WITH MY PARTNER HOW DO I PROCEED WITH DOING THAT', "FONDERING HOW I'D SET UP A JOIN TO HELL T WITH MY WIFE AND WHERE THE AP MIGHT BE", "I I'D LIKE TOY SET UP A JOINT ACCOUNT WITH MY PARTNER I'M NOT SEEING THE OPTION TO DO IT ON THE APSO I CALLED IN TO GET SOME HELP CAN I JUST DO IT OVER THE PHONE WITH YOU AND GIVE YOU THE INFORMATION OR SHOULD I DO IT IN THE AP AN I'M MISSING SOMETHING UQUETTE HAD PREFERRED TO JUST DO IT OVER THE PHONE OF POSSIBLE THINGS", 'HOW DO I FURN A JOINA COUT']


# Use another model from the Hub

In [5]:
# pipeline() can accommodate any model from the Huggingface Hub; just use the tags on the Hub to filter for an appropriate model.
# easy to adapt the pipeline() for other use-cases
# example: find model capable of handling French text for sentiment analysis
# The top filtered result returns a multilingual BERT model finetuned for sentiment analysis 
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
classifier("Nous sommes très heureux de vous présenter la bibliothèque 🤗 Transformers.")
# NB: If you can’t find a model for your use-case, you’ll need to finetune a pretrained model on your data. 
# Please consider sharing the model with the community on the Hub to democratise machine learning for everyone!

[{'label': '5 stars', 'score': 0.7272651791572571}]

# AutoClass

In [6]:
# AutoModelForSequenceClassification and AutoTokenizer classes work together to power the pipeline()
# AutoClass is a shortcut that automatically retrieves the architecture of a pretrained model from its name or path
# One only needs to select the appropriate AutoClass for task + it’s associated preprocessing class
# example based on the model used above
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)

encoding = tokenizer("Ye could have been anywhere in the world, but you're here with me. I appreciate thee!")
print(encoding)

{'input_ids': [101, 10953, 12296, 10574, 10662, 76581, 10104, 10103, 10228, 117, 10502, 10855, 112, 11449, 14048, 10171, 10525, 119, 151, 35821, 51725, 43982, 93918, 106, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [7]:
# tokenizer can also accept a list of inputs, and pad and truncate the text to return a batch with uniform length
pt_batch = tokenizer(
    ["Ye could have been anywhere in the world, but you're here with me. I appreciate thee!"],
    padding=True,
    truncation=True,
    max_length=5,
    return_tensors="pt",
)

print(pt_batch)

{'input_ids': tensor([[  101, 10953, 12296, 10574,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}


In [8]:
# Note that Autoclass was used with Autotokenizer in the French sentiment analysis example
# Transformers provides a simple and unified way to load pretrained instances
# ...each Autotokenizer has a correct Autoclass that needs to be selected for modelling
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
pt_model = AutoModelForSequenceClassification.from_pretrained(model_name)

# ...however an AutoModel can be called directly on batch input as shown below
pt_outputs = pt_model(**pt_batch)
# The model outputs the final activations in the logits attribute
# Apply the softmax function to the logits to retrieve the probabilities:
pt_predictions = torch.nn.functional.softmax(pt_outputs.logits, dim=-1)
print(pt_predictions)

tensor([[0.1918, 0.2779, 0.3638, 0.1107, 0.0558]], grad_fn=<SoftmaxBackward0>)


# Saving a model

In [9]:
# A model can be saved after fine-tuning
# Tokenizers can be saved separately
current_dir = os.getcwd()
tokenizer.save_pretrained(current_dir)
pt_model.save_pretrained(current_dir)

In [10]:
# Reload saved model
pt_model_reload = AutoModelForSequenceClassification.from_pretrained(current_dir)

In [11]:
# Transformers feature the ability to save a model and reload it as either a PyTorch or TensorFlow model
# The example here reloads a PyTorch model as a TensorFlow model using from_pt=True
from transformers import TFAutoModel
from transformers import TFAutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained(current_dir)
tf_model = TFAutoModelForSequenceClassification.from_pretrained(current_dir, from_pt=True)

2023-12-01 05:44:42.490105: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
All PyTorch model weights were used when initializing TFBertForSequenceClassification.

All the weights of TFBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


# AutoConfig for custom model builds

The model’s configuration class can be modified to change how a model is built. The configuration specifies a model’s attributes, such as the number of hidden layers or attention heads. You start from scratch when you initialize a model from a custom configuration class. The model attributes are randomly initialized, and you’ll need to train the model before you can use it to get meaningful results.

In [12]:
# import AutoConfig, and then load the pretrained model you want to modify 
# Within AutoConfig.from_pretrained(), you can specify the attribute you want to change
# e.g. the number of attention heads
from transformers import AutoConfig

custom_config = AutoConfig.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment", n_heads=12)
custom_model = AutoModel.from_config(custom_config)